<a href="https://colab.research.google.com/github/cagBRT/timeSeries/blob/main/9c_MultiHeaded_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone -l -s https://github.com/cagBRT/timeSeries.git cloned-repo
%cd cloned-repo

In [ ]:
def split_sequences(sequences, n_steps): 
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences): break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1] 
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers.merge import concatenate

# **MultiHeaded MLP**

In the last notebook we flattened the time series before entering the data into the model. <br>
Flattening the data requires will collect all the time step data, flatten the data, then use it. <br>


A second method, called MultiHeaded MLPs, has a model for each input series. <br>
Each model output is combined by another MLP to predict the output sequence.

In [ ]:
from IPython.display import Image
Image("MultiHeaded.png" , width=640)

**Define the data sequence**

In [ ]:
in_sequence1=array([10,20,30,40,50,60,70,80,90])
in_sequence2=array([15,25,35,45,55,65,75,85,95])
out_seq=array([in_sequence1[i]+in_sequence2[i] 
               for i in range(len(in_sequence1))])

In [ ]:
out_seq

Reshape the sequence

In [ ]:
in_seq1 =in_sequence1.reshape((len(in_sequence1),1))
in_seq2 =in_sequence2.reshape((len(in_sequence2),1))
out_seq =out_seq.reshape((len(out_seq),1))
dataset = hstack((in_seq1, in_seq2, out_seq))

In [ ]:
n_steps = 3

**Spilt the data into two different sets, one for each input model**

In [ ]:
X, y = split_sequences(dataset, n_steps)
X1 = X[:, :, 0]
X2 = X[:, :, 1]

**Create two input models**

In [ ]:
# first input model
visible1 = Input(shape=(n_steps,))
dense1 = Dense(100, activation='relu')(visible1)
# second input model
visible2 = Input(shape=(n_steps,))
dense2 = Dense(100, activation='relu')(visible2)

**Merge the two input models**

In [ ]:
merge = concatenate([dense1, dense2])
output = Dense(1)(merge)
model = Model(inputs=[visible1, visible2], outputs=output) 
model.compile(optimizer='adam', loss='mse')

Train the model

In [ ]:
model.fit([X1, X2], y, epochs=2000, verbose=0)

Make a prediction with new data

In [ ]:
x_input = array([[80, 85], [90, 95], [100, 105]])
x1 = x_input[:, 0].reshape((1, n_steps))
x2 = x_input[:, 1].reshape((1, n_steps))
yhat = model.predict([x1, x2], verbose=0)
print(yhat)